In [ ]:
"""
BACKGROUND: 
In VAE exercise for DL course, the variational free energy loss function (aka ELBO) has 2 terms: (1) the reconstruction
error of the decoder, and (2) the KL divergence between our approximate posterior (aka z or latent distribution) and 
the prior we provide. 


CONNECTION WITH BAYES BY BACKPROP:
Just as the VAE loss function optimizes for the parameters of the latent distribution (while keeping the reconstruction
error at bay) the loss function in Blundell (Bayes by Backprop loss) optimizes for the parameters (\mu, \sigma) that
define the weight distribution of our neural network (while keeping accuracy error at bay). 

Therefore, to implement Bayes by Backprop, we simply have to borrow the VAE loss function - this will allow us to 
approximate the true distribution of the neural network's weights. 


DATA BATCHING:
If our training set contains M number of samples (i.e. sentences in our case) and B number of batches (called 
mini-batches in Fortunato et al 2017), then we will have C = M/B number of samples per batch. Further, let's assume
that each sample/sentence contains 35 elements/words. 

BAYES BY BACKPROP THROUGH TIME:
I think it's easier to visualize with the unrolled RNN. Let's say that we unroll for 35 time steps (T=35). What this
means is that every word in a sentence corresponds to a time step (since the task is next-word prediction). Let's follow 
the path of a sample/sentence. Assume this sentence, "The dinosaur likes to go dancing." The element "The" will go to
LSTM_t1; the element "dinosaur" will go to LSTM_t2; and so on. 

I'M IN DOUBT HERE: At each timestep we will also have a y-output predicting what the next word will be in the form of
a 10K softmax vector. This means that backpropagation will be computed per word. I.e., we will forward propagate a word
through the network, make a next-word prediction, backpropagate given that prediction, and thus obtain gradients per 
word (and equivalently, per timestep). 

OPTIMIZATION:
Our direction of descent will be the average of the gradients of the C samples per batch. The gradient of each sample,
in turn, will be the average of the gradients of the 35 timesteps (i.e. the average of the 35 words). This weighting 
scheme is used by Fortunato; Blundell experiment with another that might yield better results. 

Either way, the main take-away is that in order to implement Bayes by Backprop through Time, we need to: 
    1. Figure out how to get gradients at the word (or timestep) level in tensorflow
    2. Tell tensorflow how to weight/average these gradients per step direction.
    
"""



#M = number of total samples in our training set 
batch_size = config.batch_size
n_batches = M / float(batch_size)

In [ ]:
# computing cross entropy per sample
def categorical_cross_entropy(p, t, eps=1e-10):
    return -tf.reduce_sum(t * tf.log(p+eps), axis=[1])


# computing kl divergence per sample or batch?
def kl_normal2_stdnormal(mean, log_var, eps=0.0):
    """
    Compute analytically integrated KL-divergence between a diagonal covariance Gaussian and 
    a standard Gaussian.

    In the setting of the variational autoencoder, when a Gaussian prior and diagonal Gaussian 
    approximate posterior is used, this analytically integrated KL-divergence term yields a lower variance 
    estimate of the likelihood lower bound compared to computing the term by Monte Carlo approximation.

        .. math:: D_{KL}[q_{\phi}(z|x) || p_{\theta}(z)]

    See appendix B of [KINGMA]_ for details.

    Parameters
    ----------
    mean : Tensorflow tensor
        Mean of the diagonal covariance Gaussian.
    log_var : Tensorflow tensor
        Log variance of the diagonal covariance Gaussian.

    Returns
    -------
    Tensorflow tensor
        Element-wise KL-divergence, this has to be summed when the Gaussian distributions are multi-variate.

    References
    ----------
        ..  [KINGMA] Kingma, Diederik P., and Max Welling.
            "Auto-Encoding Variational Bayes."
            arXiv preprint arXiv:1312.6114 (2013).

    """
    return -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean) - tf.exp(log_var), axis=1)


c = - 0.5 * math.log(2*math.pi)
def log_normal2(x, mean, log_var, eps=0.0):
    """
    Compute log pdf of a Gaussian distribution with diagonal covariance, at values x.
    Here variance is parameterized in the log domain, which ensures :math:`\sigma > 0`.

        .. math:: \log p(x) = \log \mathcal{N}(x; \mu, \sigma^2I)
    
    Parameters
    ----------
    x : Tensorflow tensor
        Values at which to evaluate pdf.
    mean : Tensorflow tensor
        Mean of the Gaussian distribution.
    log_var : Tensorflow tensor
        Log variance of the diagonal covariance Gaussian.
    eps : float
        Small number used to avoid NaNs

    Returns
    -------
    Tensorflow tensor
        Element-wise log probability, this has to be summed for multi-variate distributions.
    """
    return tf.reduce_sum(c - log_var/2 - tf.square(x - mean) / (2 * tf.exp(log_var) + eps), axis=[1])

In [ ]:
eps = 1e-10

# Loss Function Based on Blundell et al 2015 Equation 1

#______ Version 1: Analytical Closed form solution _______#
# Likelihood cost (accuracy error), log[p(D|w)] in [-\infty, 0]). 
log_p_D_given_w = -tf.reduce_mean(categorical_cross_entropy(l_out, tf.expand_dims(x_pl,1), eps=eps), name = "log_likelihood")

# Regularization cost: 
# Kulback-Leibler divergence between approximate posterior, q(w|\theta)
# and prior p(w)=N(w,mu,sigma*I). In this case we use a gaussian prior - not a mixture
KL_qp = tf.reduce_mean(kl_normal2_stdnormal(l_mu_q, l_logvar_q, eps=eps), name = "KL")

# Combining the two terms in the variational free energy (aka ELBO): Blundell Eq1   
ELBO_loss = tf.subtract(KL_qp, log_p_x_given_z, name="ELBO")


#______ Version 2: Approximation via Monte Carlo _______#
# This might be useful if we figure out how to implement with gaussian mixtures as priors
#ELBO_loss = ((1. / n_batches) * (log_qw - log_pw) - log_likelihood).sum() / float(batch_size)



# Optimizer
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.25)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

# Training operator for applying the loss gradients in backpropagation update
train_op = optimizer.minimize(ELBO_loss)